In [52]:
import cv2 
import numpy as np

upscale = 10
def plotPaths(paths):
    img = np.ones((297 * upscale, 420 * upscale, 3), np.uint8) * 255

    for path in paths :
        for i in range(len(path)-1):
            a = (round(path[i][0]* upscale) , round(path[i][1]* upscale) )
            b = (round(path[i+1][0]* upscale) , round(path[i+1][1]* upscale) )
            cv2.line(img, a, b, (0,0,0), 1, cv2.LINE_AA)

    cv2.imwrite("render.png", img)


def circle_path(pos, radius, num_points=6):
    angles = np.linspace(0, 2 * np.pi, num_points)
    path = radius * np.array([np.cos(angles), np.sin(angles)]).T + pos;
    return np.array(path)


paths = []



// voronoi  relaxation

//grid based hatching

// units are in mm

In [60]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d, KDTree
from tqdm import tqdm
from random import random
from math import sqrt 


def plotPoints(points):
    paths = [circle_path(p/4, 0.1) for p in points]
    plotPaths(paths)

# Load image
image_path = "/Users/nshelton/Pictures/cat.jpg"

image = cv2.imread(image_path)
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# image_value = cv2.resize(image_gray, (-1, -1), fx = 0.5, fy = 0.5)
image_value = image_gray
image_value = (256 - image_value).astype(np.float32)
image_value /= 255

h,w = image_value.shape
print(w,h)

points = []
n_points = 10000
max_tries = 10000000
i = 0

with tqdm(total=n_points) as pbar:
    while len(points) < n_points:
        i += 1
        if i > max_tries :
            break

        x, y = random() * w, random() * h
        if image_value[int(y),(int(x))] > (random()):
            pbar.update(1)
            points.append([x,y])

    points = np.array(points)

plotPoints(points)


for i in range(10):
    tree = KDTree(points)

    points_to_centroid = np.zeros((len(points), 2))
    points_to_totals = np.zeros((len(points), 1))

    for row in tqdm(range(image_value.shape[0])):
        for col in range(image_value.shape[1]):
            px = (col, row)
            value = image_value[row, col]
            d, idx = tree.query(px)
            points_to_centroid[idx] += np.array(px) * value
            points_to_totals[idx] += value

    # remove useless points (white areas)
    mask, _  = np.nonzero(points_to_totals)
    
    points = points_to_centroid[mask]/points_to_totals[mask]


    plotPoints(points)




736 1051


 26%|██▌       | 269/1051 [00:03<00:10, 76.55it/s]


KeyboardInterrupt: 

In [38]:

print(points_to_totals.shape)
print(mask.shape)

(10000, 1)
(9966,)


In [62]:
 
import json
import uuid

projects_folder = "/Users/nshelton/Hephaestus/projects"

def writePlot(paths):
    dictionary = {
        "created_time": "4/7/2024 4:42:27 PM",
        "modified_time": "4/7/2024 6:27:36 PM",
        "camera_position": [100, 100],
        "zoom": 100.702,
        "aspect": 0.563,
        "dom_element": {},
        "plot_models": [],
    }

    dictionary["plot_models"].append(
        {
            "position": {"x": 0, "y": 0},
            "paths": np.array(paths).tolist(),
            "scale": 0.1,
            "id": str(uuid.uuid4()),
            "state": "none",
        }
    )

    with open(projects_folder + "/out.json", "w") as outfile:
        json.dump(dictionary, outfile)

paths = [circle_path(p, 1) for p in points]

writePlot(paths)

